In [ ]:
import copy
import numpy as np
import random

In [ ]:
class Checkers:
  BOARD_SIZE = 8

  def __init__(self):
      self.SYMBOLS = ('R','B')
      self.EXP_COUNT = 10
      self.weight_vec = [np.random.rand(7), np.random.rand(7)]
      self.game_status_count = [0, 0, 0]

      for _ in range(self.EXP_COUNT):
          initial_board = self._init_board()
          game_history = self._get_history(self.weight_vec[0], self.weight_vec[1], initial_board)
          p1_training_data = self._get_samples(self.weight_vec[0], self.SYMBOLS[0], self.SYMBOLS[1], game_history)
          p2_training_data = self._get_samples(self.weight_vec[1], self.SYMBOLS[1], self.SYMBOLS[0], game_history)
          self.game_status_count = self._get_status_count(self.SYMBOLS[0], self.SYMBOLS[1], game_history)
          self.weight_vec = [self._lms(self.weight_vec[0], p1_training_data), self._lms(self.weight_vec[1], p1_training_data)]

      print("\nResults of games: (" + "Player-1 Won " + str(self.game_status_count[0]) +
            " times, Player-2 Won " + str(self.game_status_count[1]) + " times, Draws " + 
            str(self.game_status_count[2]) + " times)\n"
      )

      self.weights = list(np.mean(np.array([self.weight_vec[0], self.weight_vec[1]]), axis=0))
      print("Vector after learning: \n" + str(self.weights))

  def _init_board(self):
      board = [[' ' for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
      nRows = (self.BOARD_SIZE//2)-1
      for i in range(nRows):
          if (i % 2 == 0):
              start = 1
          else:
              start = 0
          for j in range(start, self.BOARD_SIZE, 2):
              board[i][j] = 'B'
      for i in range(self.BOARD_SIZE-1, self.BOARD_SIZE-1-nRows, -1):
          if (i % 2 == 0):
              start = 1
          else:
              start = 0
          for j in range(start, self.BOARD_SIZE, 2):
              board[i][j] = 'R'
      return board
  
  def _get_inter_score(self, weightVector, featureVector):
      weightVector = np.array(weightVector).reshape((len(weightVector), 1))
      featureVector = np.array(featureVector).reshape((len(featureVector), 1))
      boardScore = np.dot(weightVector.T, featureVector)
      return (boardScore[0][0])

  def _choose_move(self, board, p1, p2, weightVector):
      legalMoves = self._get_legal_moves(board, p1)
      if (len(legalMoves) == 0):
          return -1
      legalMoveScores = [self._get_inter_score(weightVector,
                        self._get_features(i, p1, p2)) for i in legalMoves]
      return legalMoves[np.argmax(legalMoveScores)]

  def _choose_rand_move(self, board, symbol):
      legalMoves = self._get_legal_moves(board, symbol)
      if (len(legalMoves) == 0):
          return -1
      return random.choice(legalMoves)

  def _get_history(self, weightVector1, weightVector2, board):
      game_history = []
      gameStatusFlag = True

      tempBoard = copy.deepcopy(board)
      while (gameStatusFlag):
          tempBoard = self._choose_move(tempBoard, self.SYMBOLS[0], self.SYMBOLS[1], weightVector1)
          if (tempBoard == -1):
              break
          game_history.append(tempBoard)
          gameStatusFlag = not self._is_game_over(tempBoard, self.SYMBOLS[0])
          if (gameStatusFlag == False):
              break
          tempBoard = self._choose_rand_move(tempBoard, self.SYMBOLS[1])
          if (tempBoard == -1):
              break
          game_history.append(tempBoard)
          gameStatusFlag = not self._is_game_over(tempBoard, self.SYMBOLS[1])
      return game_history

  def _get_final_score(self, board, p1, p2):
      score = 0
      if (self._is_game_over(board, p1)):
          score = 100
      elif (self._is_game_over(board, p2)):
          score = -100

      return score

  def _get_samples(self, weightVector, p1, p2, game_history):
      trainingData = []
      for i in range(len(game_history)-1):
          featureVector = self._get_features(game_history[i+1], p1, p2)
          trainingData.append([featureVector, self._get_inter_score(weightVector, featureVector)])
      trainingData.append([self._get_features(game_history[-1], p1, p2),
                           self._get_final_score(game_history[-1], p1, p2)])
      return trainingData

  def _get_status_count(self, p1, p2, game_history):
      finalScore = self._get_final_score(game_history[-1], p1, p2)
      if (finalScore == 100):
          self.game_status_count[0] += 1
      elif (finalScore == -100):
          self.game_status_count[1] += 1
      else:
          self.game_status_count[2] += 1
      return self.game_status_count

  def _lms(self, weightVector, trainData, lRate=0.01):
      for t in trainData:
          vTrainBoardState = t[1]
          vHatBoardState = self._get_inter_score(weightVector, t[0])
          weightVector = weightVector + lRate * (vTrainBoardState - vHatBoardState) * np.array(t[0])
      return weightVector

  def _can_move(self, board, symbol, r1, c1, r2, c2):
      if r2 < 0 or r2 >= self.BOARD_SIZE or c2 < 0 or c2 >= self.BOARD_SIZE:
          return False

      if board[r2][c2] != ' ':
          return False

      if symbol == 'R':
          if board[r1][c1] == 'R' and r2 > r1:
              return False
          return True

      else:
          if board[r1][c1] == 'B' and r2 < r1:
              return False
          return True

  def _can_jump(self, board, symbol, r1, c1, r2, c2, r3, c3):
      if r3 < 0 or r3 >= self.BOARD_SIZE or c3 < 0 or c3 >= self.BOARD_SIZE:
          return False
      if board[r3][c3] != ' ':
          return False
      if symbol == 'R':
          if board[r1][c1] == 'R' and r3 > r1:
              return False
          if board[r2][c2] != 'B' and board[r2][c2] != 'KB':
              return False
          return True
      else:
          if board[r1][c1] == 'B' and r3 < r1:
              return False
          if board[r2][c2] != 'R' and board[r2][c2] != 'KR':
              return False
          return True

  def _make_kings(self, board):
    for j in range(self.BOARD_SIZE):
      if board[0][j] == 'R':
          board[0][j] = 'KR'
      if board[self.BOARD_SIZE-1][j] == 'B':
          board[self.BOARD_SIZE-1][j] = 'KB'

  def _get_legal_moves(self, boardState, symbol):
      legalMoves = []

      if symbol == 'B':
          playerKing = 'KB'
      else:
          playerKing = 'KR'
      for i in range(len(boardState[0])):
          for j in range(len(boardState[0])):
              if boardState[i][j] == symbol or boardState[i][j] == playerKing:
                  if self._can_jump(boardState, symbol, i, j, i+1, j+1, i+2, j+2):
                      tempBoard = copy.deepcopy(boardState)
                      tempBoard[i+2][j+2] = tempBoard[i][j]
                      tempBoard[i][j] = ' '
                      tempBoard[i+1][j+1] = ' '
                      self._make_kings(boardState)
                      legalMoves.append(tempBoard)
                  if self._can_jump(boardState, symbol, i, j, i-1, j+1, i-2, j+2):
                      tempBoard = copy.deepcopy(boardState)
                      tempBoard[i-2][j+2] = tempBoard[i][j]
                      tempBoard[i][j] = ' '
                      tempBoard[i-1][j+1] = ' '
                      self._make_kings(boardState)
                      legalMoves.append(tempBoard)
                  if self._can_jump(boardState, symbol, i, j, i+1, j-1, i+2, j-2):
                      tempBoard = copy.deepcopy(boardState)
                      tempBoard[i+2][j-2] = tempBoard[i][j]
                      tempBoard[i][j] = ' '
                      tempBoard[i+1][j-1] = ' '
                      self._make_kings(boardState)
                      legalMoves.append(tempBoard)
                  if self._can_jump(boardState, symbol, i, j, i-1, j-1, i-2, j-2):
                      tempBoard = copy.deepcopy(boardState)
                      tempBoard[i-2][j-2] = tempBoard[i][j]
                      tempBoard[i][j] = ' '
                      tempBoard[i-1][j-1] = ' '
                      self._make_kings(boardState)
                      legalMoves.append(tempBoard)
      if len(legalMoves) == 0:
          for i in range(len(boardState[0])):
              for j in range(len(boardState[0])):
                  if boardState[i][j] == symbol or boardState[i][j] == playerKing:
                      if self._can_move(boardState, symbol, i, j, i+1, j+1):
                          tempBoard = copy.deepcopy(boardState)
                          tempBoard[i+1][j+1] = tempBoard[i][j]
                          tempBoard[i][j] = ' '
                          self._make_kings(boardState)
                          legalMoves.append(tempBoard)
                      if self._can_move(boardState, symbol, i, j, i-1, j+1):
                          tempBoard = copy.deepcopy(boardState)
                          tempBoard[i-1][j+1] = tempBoard[i][j]
                          tempBoard[i][j] = ' '
                          self._make_kings(boardState)
                          legalMoves.append(tempBoard)
                      if self._can_move(boardState, symbol, i, j, i+1, j-1):
                          tempBoard = copy.deepcopy(boardState)
                          tempBoard[i+1][j-1] = tempBoard[i][j]
                          tempBoard[i][j] = ' '
                          self._make_kings(boardState)
                          legalMoves.append(tempBoard)
                      if self._can_move(boardState, symbol, i, j, i-1, j-1):
                          tempBoard = copy.deepcopy(boardState)
                          tempBoard[i-1][j-1] = tempBoard[i][j]
                          tempBoard[i][j] = ' '
                          self._make_kings(boardState)
                          legalMoves.append(tempBoard)

      return legalMoves

  def _get_features(self, board, p1, p2):
      x = np.zeros(7)
      x[0] = 1
      for i in range(self.BOARD_SIZE):
          for j in range(self.BOARD_SIZE):
              if board[i][j] == p1:
                  x[1] += 1
              elif board[i][j] == p2:
                  x[2] += 1
              elif board[i][j] == ('K'+p1):
                  x[3] += 1
              elif board[i][j] == ('K'+p2):
                  x[4] += 1
              if board[i][j] == p1 or board[i][j] == ('K'+p1):
                  if self._can_jump(board, p1, i, j, i+1, j+1, i+2, j+2):
                      x[5] += 1
                  if self._can_jump(board, p1, i, j, i-1, j+1, i-2, j+2):
                      x[5] += 1
                  if self._can_jump(board, p1, i, j, i+1, j-1, i+2, j-2):
                      x[5] += 1
                  if self._can_jump(board, p1, i, j, i-1, j-1, i-2, j-2):
                      x[5] += 1
              elif board[i][j] == p2 or board[i][j] == ('K'+p2):
                  if self._can_jump(board, p2, i, j, i+1, j+1, i+2, j+2):
                      x[6] += 1
                  if self._can_jump(board, p2, i, j, i-1, j+1, i-2, j+2):
                      x[6] += 1
                  if self._can_jump(board, p2, i, j, i+1, j-1, i+2, j-2):
                      x[6] += 1
                  if self._can_jump(board, p2, i, j, i-1, j-1, i-2, j-2):
                      x[6] += 1
      return x

  def _is_game_over(self, board, symbol):
      for i in range(self.BOARD_SIZE):
          for j in range(self.BOARD_SIZE):
              if (board[i][j] != symbol):
                  return 0
      return 1

  def print_board(self, board):
      for _ in range(6*self.BOARD_SIZE):
          print("-", end='')
      print("")
      for i in range(self.BOARD_SIZE):
          print('|  ', end='')
          for j in range(self.BOARD_SIZE):
              print(board[i][j], end='  |  ')
          print()
          for _ in range(6*self.BOARD_SIZE):
              print("-", end='')
          print()


  def play(self):
      print("\nReady? (y/n)")
      ans = input()
      while ans == "y":
          boardState = self._init_board()
          gameStatusFlag = True
          game_history = []
          print("\nYou are player RED\n")
          self.print_board(boardState)
          while (gameStatusFlag):
              print('Your Turn:\n')
              print('coordinates (x, y)')
              x, y = list(map(int, input().strip().split(' ')))
              print('l/L to move left OR r/R to move right: ')
              dir = input()
              boardState[x][y] = ' '
              if dir == 'l' or dir == 'L':
                  if (self._can_jump(boardState, 'R', x, y, x-1, y-1, x-2, y-2)):
                      boardState[x-2][y-2] = 'R'
                      boardState[x-1][y-1] = ' '
                      boardState[x][y] = ' '
                  else:
                      boardState[x-1][y-1] = 'R'
                      boardState[x][y] = ' '
              else:
                  if (self._can_jump(boardState, 'R', x, y, x-1, y+1, x-2, y+2)):
                      boardState[x-2][y+2] = 'R'
                      boardState[x-1][y+1] = ' '
                      boardState[x][y] = ' '
                  else:
                      boardState[x-1][y+1] = 'R'
                      boardState[x][y] = ' '
              self._make_kings(boardState)
              self.print_board(boardState)
              game_history.append(boardState)
              gameStatusFlag = not self._is_game_over(boardState, 'R')
              if (gameStatusFlag == False):
                  break
              boardState = self._choose_move(boardState, 'B', 'R', self.weights)
              print('Computers\'s Turn:\n')
              self.print_board(boardState)
              game_history.append(boardState)
              gameStatusFlag = not self._is_game_over(boardState, 'B')

          if (self._is_game_over(boardState, 'B')):
              print("AI Won")
          elif (self._is_game_over(boardState, 'R')):
              print("You Won")
          else:
              print("Draw")
          print("Play again?")
          ans = input()


In [ ]:
checkers = Checkers()
checkers.play()


Results of games: (#Games Player-1 Wins = 0, #Games Player-2 Wins = 0, Games Drawn = 10)

Vector after learning: 
[-155.8954966196702, -1032.6530566468198, 122.82646002692726, 0.33616914807023546, -144.99020171195613, 1835.9687676588555, -2098.918294931346]

Ready? (y/n)

You are player RED

------------------------------------------------
|     |  B  |     |  B  |     |  B  |     |  B  |  
------------------------------------------------
|  B  |     |  B  |     |  B  |     |  B  |     |  
------------------------------------------------
|     |  B  |     |  B  |     |  B  |     |  B  |  
------------------------------------------------
|     |     |     |     |     |     |     |     |  
------------------------------------------------
|     |     |     |     |     |     |     |     |  
------------------------------------------------
|  R  |     |  R  |     |  R  |     |  R  |     |  
------------------------------------------------
|     |  R  |     |  R  |     |  R  |     |  R  |  